## Transfer learning to recognise new pictures

In [1]:
import logging
import tensorflow as tf
import keras

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
log = logging.getLogger()

%config Completer.use_jedi = False # make autocompletion works in jupyter

tf.__version__

'2.4.1'

In [1]:
import argparse 


args = argparse.Namespace()
args.train_full_folder = './transfer-pic/'
args.train_full_car_folder = args.train_full_folder + 'car/'
args.train_full_no_car_folder = args.train_full_car_folder + 'no-car/' 


